# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt

import os
import io
import boto3
import sagemaker
from sagemaker import get_execution_role

% matplotlib inline

In [2]:
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Name of directory created to save your features data
data_dir = 'starbucks_data'

# Set prefix, a descriptive name for a directory  
prefix = 'capstone'

# Upload dataset to S3
train_location = sagemaker_session.upload_data(os.path.join(data_dir, 'train.csv'), key_prefix=prefix)
valid_location = sagemaker_session.upload_data(os.path.join(data_dir, 'valid.csv'), key_prefix=prefix)
test_location = sagemaker_session.upload_data(os.path.join(data_dir, 'test.csv'), key_prefix=prefix)

# Initialize the default bucket to store the trained model artifacts later
bucket = sagemaker_session.default_bucket()
print(bucket)

sagemaker-us-east-2-634441263192


In [3]:
# Load data from local directory
# Needed for LinearLearner built-in model
train = pd.read_csv('starbucks_data/train.csv', header=None)
X_train = train.drop([0], axis=1).to_numpy('float32')
y_train = train[0].to_numpy('float32')

valid = pd.read_csv('starbucks_data/valid.csv', header=None)
X_valid = valid.drop([0], axis=1).to_numpy('float32')
y_valid = valid[0].to_numpy('float32')

test = pd.read_csv('starbucks_data/test.csv', header=None)
X_test = test.drop([0], axis=1).to_numpy('float32')
y_test = test[0].to_numpy('float32')

In [4]:
# Use this cell if NOT using HyperparameterTuner
# Concatenate train and validation set into bigger trainig set
# Linear Learner does not take validation set as input
X_train = np.concatenate((X_train, X_valid))
y_train = np.concatenate((y_train, y_valid))

In [5]:
# Import LinearLearner
from sagemaker import LinearLearner

# Specify an output path
prefix = 'capstone-starbucks'
output_path = 's3://{}/{}'.format(bucket, prefix)

# Instantiate LinearLearner
linear = LinearLearner(
    role=role,
    train_instance_count=1, 
    train_instance_type='ml.c4.xlarge',
    predictor_type='regressor',
    output_path=output_path,
    sagemaker_session=sagemaker_session,
    optimizer='sgd',
    epochs=100,
    num_calibration_samples=3000,
    #learning_rate=0.01,
    #loss='rmse',
    #early_stopping_patience=3,
    #early_stopping_tolerance=0.005
)

In [6]:
# Create RecordSet
formatted_train_data = linear.record_set(train=X_train, labels=y_train)
#formatted_valid_data = linear.record_set(train=X_valid, labels=y_valid, channel='validation')
#formatted_test_data = linear.record_set(train=X_test, labels=y_test, channel='test')

In [7]:
# Train the model on the training data
linear.fit(formatted_train_data)

2020-04-02 00:15:22 Starting - Starting the training job...
2020-04-02 00:15:24 Starting - Launching requested ML instances............
2020-04-02 00:17:25 Starting - Preparing the instances for training......
2020-04-02 00:18:53 Downloading - Downloading input data
2020-04-02 00:18:53 Training - Downloading the training image...
2020-04-02 00:19:24 Uploading - Uploading generated training model
2020-04-02 00:19:24 Completed - Training job completed
Docker entrypoint called with argument(s): train
Running default environment configuration script
[04/02/2020 00:19:09 INFO 140367033890624] Reading default configuration from /opt/amazon/lib/python2.7/site-packages/algorithm/resources/default-input.json: {u'loss_insensitivity': u'0.01', u'epochs': u'15', u'feature_dim': u'auto', u'init_bias': u'0.0', u'lr_scheduler_factor': u'auto', u'num_calibration_samples': u'10000000', u'accuracy_top_k': u'3', u'_num_kv_servers': u'auto', u'use_bias': u'true', u'num_point_for_scaler': u'10000', u'_log_

Training seconds: 54
Billable seconds: 54


In [8]:
%%time 
# Deploy and create a predictor
linear_predictor = linear.deploy(initial_instance_count=1, instance_type='ml.t2.medium')

---------------!CPU times: user 257 ms, sys: 8.53 ms, total: 266 ms
Wall time: 7min 31s


In [9]:
from sklearn.metrics import r2_score
import ast

# code to evaluate the endpoint on test data
# returns the r2 score of the model
def evaluate(predictor, X, y):
    """
    Evaluate a model on a test set given the prediction endpoint.  
    Return r2 score regression metric.
    :param predictor: A prediction endpoint
    :param test_features: Test features
    :param test_labels: Value labels for test data
    :return: R2 score of the mode.
    """
    # LinearLearner produces a `score` for each data point
    predictions = predictor.predict(X)
    y_pred = np.concatenate(([x.label['score'].float32_tensor.values for x in predictions]))
    return r2_score(y, y_pred)

In [10]:
r2_score_train = evaluate(linear_predictor, X_train, y_train)
print(r2_score_train)

0.09343373445065872


In [11]:
r2_score = evaluate(linear_predictor, X_test, y_test)
print(r2_score)

0.08978870096576208


In [16]:
performance_drop = (r2_score_train - r2_score) / r2_score_train
print('{}%'.format(round(performance_drop,3)*100))

3.9%


In [13]:
# Deletes a precictor.endpoint
def delete_endpoint(predictor):
    try:
        boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
        print('Deleted {}'.format(predictor.endpoint))
    except:
        print('Already deleted: {}'.format(predictor.endpoint))

# delete the predictor endpoint 
delete_endpoint(linear_predictor)

Deleted linear-learner-2020-04-02-00-15-22-476
